In [1]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from static import QuadcopterForceMoments
from vlm import VLM
from vpm_cuda import VPM
from wind import WindField
import numpy as np
import matplotlib.pyplot as plt
import numba as nb
import copy
import pyvista as pv
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.gridspec import GridSpec
import pandas as pd

#-----------------------------------------------------------------------------------------------------------------------------------------------------------------_#
wake_states = []  # List to store states at each time step

# Initialize geometry and mesh
# propeller_geometry = PropellerGeometry(
#     airfoil_distribution_file='DJI9443_airfoils.csv',
#     chorddist_file='DJI9443_chorddist.csv',
#     pitchdist_file='DJI9443_pitchdist.csv',
#     sweepdist_file='DJI9443_sweepdist.csv',
#     heightdist_file='DJI9443_heightdist.csv',
#     R_tip=0.11938,
#     R_hub=0.00624,
#     num_blades=2
# )

propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='NASA_UAM_quadrotor.csv',
    chorddist_file='NASA_UAM_quadrotor_chorddist.csv',
    pitchdist_file='NASA_UAM_quadrotor_pitchdist.csv',
    sweepdist_file='NASA_UAM_quadrotor_sweepdist.csv',
    R_tip=2.809,
    R_hub=0.3372,
    num_blades=3
)


propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=0.175, com=(0, 0, 0))
print('Propeller Loaded Succesfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Succesfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg
inertia_matrix = np.array([
    [1.0, 0, 0],
    [0, 1.0, 0],
    [0, 0, 1.0]

])  # Simple diagonal inertia matrix for example

initial_position = np.array([0, 0, 0])  # Starting position
initial_body_velocity = np.array([0, 0, 0])  # Starting velocity
initial_orientation = np.array([0, 0, 0])  # Starting roll, pitch, yaw
initial_angular_velocity = np.array([0, 0, 0])  # Starting angular rates


# Initialize the UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)


wake_states = []

dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=np.zeros(3),
    initial_velocity_body=np.zeros(3),
    initial_angles=np.zeros(3),
    initial_angular_rates=np.zeros(3)
)

thrust_history = []
wake_history = []
time_points = []
normalized_forces = []
rho = 1.071778   

mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)


target_rpm = 565
n_revs = 1
n_steps_rev = 36
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (target_rpm * n_steps_rev)

probe_steps = 1


x_positions = np.linspace(500, 150, probe_steps) 
x_positions = [325]
y_positions = [135]  # Three different y-positions to test
z_pos = 50  # Constant z position



for y_pos in y_positions:
    print(f"Testing y-position: {y_pos}")
    for x_pos  in x_positions:
        com_position = np.array([x_pos, y_pos, z_pos])

        print("Probe step:", x_pos)

        for time_step in range(1, n_steps): #this is for azimuth

            print("Step per revolution:", time_step)

            rpm = 46
            omega_dict = {  # Motor speed initialization
                'Propeller_1': np.array([0, 0, rpm]),
                'Propeller_2': np.array([0, 0, -rpm]),
                'Propeller_3': np.array([0, 0, -rpm]),
                'Propeller_4': np.array([0, 0, rpm])
            }

            # Step 1: Calculate forces and moments using UVLM
            forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
                propeller_mesh=quad_propeller_mesh,
                dt=dt,
                rho=rho,
                time_step=time_step,
                body_velocity=dynamics.velocity_body, 
                omega=omega_dict,
                wind_field=wind_field,
                com_position=com_position,
                roll=dynamics.angles[0],
                pitch=dynamics.angles[1],
                yaw=dynamics.angles[2],
                
            )
                
            thrust = np.zeros(3)
            for prop_key in ['Propeller_1']:
                thrust += forces_and_moments[prop_key]['force'][2]

            print('force', forces_and_moments[prop_key]['force'])
            print('momnet', forces_and_moments[prop_key]['force'])
          
    
            #  Step 2: Update mesh geometry for propeller rotation
            SixDOFDynamics.update_mesh_transformations(
                propeller_mesh=quad_propeller_mesh,
                dynamics=dynamics,
                dt=dt,
                omega_dict=omega_dict
            )
        


Propeller Loaded Succesfully
Mesh Generated Succesfully
Testing y-position: 135
Probe step: 325
Step per revolution: 1
2.15983417290935
Step per revolution: 2
2.1627193302207934
Step per revolution: 3
2.162174665230622
Step per revolution: 4
2.1582658725750514
Step per revolution: 5
2.1514644103359792
Step per revolution: 6
2.14259063523833
Step per revolution: 7
2.1327148555218614
Step per revolution: 8
2.123028235971344
Step per revolution: 9
2.114699125863864
Step per revolution: 10
2.108732138802965
Step per revolution: 11
2.105846981491523
Step per revolution: 12
2.1063916464816943
Step per revolution: 13
2.110300439137261
Step per revolution: 14
2.117101901376335
Step per revolution: 15
2.1259756764739817
Step per revolution: 16
2.1358514561904682
Step per revolution: 17
2.145538075740986
Step per revolution: 18
2.1538671858484553
Step per revolution: 19
2.1598341729093593
Step per revolution: 20
2.1627193302208076
Step per revolution: 21
2.16217466523063
Step per revolution: 22


In [ ]:
from propeller import PropellerGeometry
from mesh import PropellerMesh
from dynamics import SixDOFDynamics
from vlm import VLM
from wind import WindField
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyvista as pv

# Initialize geometry and mesh (same as your current code)
propeller_geometry = PropellerGeometry(
    airfoil_distribution_file='DJI9443_airfoils.csv',
    chorddist_file='DJI9443_chorddist.csv',
    pitchdist_file='DJI9443_pitchdist.csv',
    sweepdist_file='DJI9443_sweepdist.csv',
    heightdist_file='DJI9443_heightdist.csv',
    R_tip=0.11938,
    R_hub=0.00624,
    num_blades=2
)

propeller_mesh_system = PropellerMesh(propeller_geometry, arm_length=0.175, com=(0, 0, 0))
print('Propeller Loaded Successfully')

quad_propeller_mesh = propeller_mesh_system.generate_quad_propeller_mesh()
print('Mesh Generated Successfully')

# Initialize SixDOF dynamics
mass = 0.68  # kg
inertia_matrix = np.array([
    [0.0045, 0, 0],
    [0, 0.0045, 0],
    [0, 0, 0.009]
])

# Initial states - using your specified hover position
initial_position = np.array([325, 135, 50])  # Your specified position
initial_body_velocity = np.array([0, 0, 0])  # Starting with zero velocity
initial_orientation = np.array([0, 0, 0])  # Level orientation
initial_angular_velocity = np.array([0, 0, 0])  # No angular velocity

# Initialize dynamics
dynamics = SixDOFDynamics(
    mass=mass,
    inertia_matrix=inertia_matrix,
    initial_position=initial_position,
    initial_velocity_body=initial_body_velocity,
    initial_angles=initial_orientation,
    initial_angular_rates=initial_angular_velocity
)

# Initialize UVLM solver
uvlm_solver = VLM(quad_propeller_mesh)

# Load wind field
mesh = pv.read(r"bp_50percent_75.vtk") 
wind_field = WindField(mesh)

# Simulation parameters
rpm = 565  # Fixed RPM
n_revs = 1  # Run for 10 revolutions to see response
n_steps_rev = 1
n_steps = n_revs * n_steps_rev
dt = 2*np.pi / (rpm * n_steps_rev)
td=0.01
# Fixed motor speeds (open loop)
omega_dict = {
    'Propeller_1': np.array([0, 0, rpm]),
    'Propeller_2': np.array([0, 0, -rpm]),
    'Propeller_3': np.array([0, 0, -rpm]),
    'Propeller_4': np.array([0, 0, rpm])
}

# History storage
time_history = []
position_history = []
velocity_history = []
orientation_history = []
force_history = []
moment_history = []
propeller_forces = []  # To store individual propeller forces

rho = 1.071778

# Open-loop simulation
print("Starting open-loop simulation...")
for time_step in range(1, 50):
    # Current time
    current_time = time_step * td
    
    # Calculate forces and moments using UVLM with current state
    forces_and_moments = uvlm_solver.calculate_total_forces_and_moments(
        propeller_mesh=quad_propeller_mesh,
        dt=dt,
        rho=rho,
        time_step=time_step,
        body_velocity=dynamics.velocity_body,
        omega=omega_dict,
        wind_field=wind_field,
        com_position=dynamics.position,  # Use current position from dynamics
        roll=dynamics.angles[0],
        pitch=dynamics.angles[1],
        yaw=dynamics.angles[2]
    )
    
    # Calculate total forces and moments
    total_force = np.zeros(3)
    total_moment = np.zeros(3)
    
    # Individual propeller forces
    prop_forces = {}
    
    for prop_key in ['Propeller_1']:
        prop_force = forces_and_moments[prop_key]['force']
        prop_moment = forces_and_moments[prop_key]['moment']
        
        total_force += prop_force
        total_moment += prop_moment
        
        prop_forces[prop_key] = prop_force
    
    # Store individual propeller forces
    propeller_forces.append(prop_forces)
    
    # Update dynamics with calculated forces and moments
    dynamics.update_states(
        forces_body=total_force,
        moments_body=total_moment,
        dt=td,
        current_time=current_time
    )
    
    # Update mesh geometry for propeller rotation
    SixDOFDynamics.update_mesh_transformations(
        propeller_mesh=quad_propeller_mesh,
        dynamics=dynamics,
        dt=dt,
        omega_dict=omega_dict
    )
    
    # Store history
    time_history.append(current_time)
    position_history.append(dynamics.position.copy())
    velocity_history.append(dynamics.velocity_body.copy())
    orientation_history.append(dynamics.angles.copy())
    force_history.append(total_force.copy())
    moment_history.append(total_moment.copy())
    
    # Print progress
    if time_step % n_steps_rev == 0:
        print(f"Completed revolution {time_step // n_steps_rev}/{n_revs}")
        print(f"Position: {dynamics.position}")
        print(f"Velocity: {dynamics.velocity_body}")
        print(f"Orientation: {dynamics.angles}")
        print(f"Total force: {total_force}")
        print(f"Total moment: {total_moment}")

# Convert history lists to numpy arrays for analysis
time_history = np.array(time_history)
position_history = np.array(position_history)
velocity_history = np.array(velocity_history)
orientation_history = np.array(orientation_history)
force_history = np.array(force_history)
moment_history = np.array(moment_history)

# Save results to CSV for later analysis
results_df = pd.DataFrame({
    'time': time_history,
    'position_x': position_history[:, 0],
    'position_y': position_history[:, 1],
    'position_z': position_history[:, 2],
    'velocity_x': velocity_history[:, 0],
    'velocity_y': velocity_history[:, 1],
    'velocity_z': velocity_history[:, 2],
    'roll': orientation_history[:, 0],
    'pitch': orientation_history[:, 1],
    'yaw': orientation_history[:, 2],
    'force_x': force_history[:, 0],
    'force_y': force_history[:, 1],
    'force_z': force_history[:, 2],
    'moment_x': moment_history[:, 0],
    'moment_y': moment_history[:, 1],
    'moment_z': moment_history[:, 2]
})

results_df.to_csv('open_loop_simulation_results.csv', index=False)

# Also save individual propeller forces
prop_forces_df = pd.DataFrame({
    'time': time_history,
    'prop1_force_x': [forces['Propeller_1'][0] for forces in propeller_forces],
    'prop1_force_y': [forces['Propeller_1'][1] for forces in propeller_forces],
    'prop1_force_z': [forces['Propeller_1'][2] for forces in propeller_forces],
    'prop2_force_x': [forces['Propeller_2'][0] for forces in propeller_forces],
    'prop2_force_y': [forces['Propeller_2'][1] for forces in propeller_forces],
    'prop2_force_z': [forces['Propeller_2'][2] for forces in propeller_forces],
    'prop3_force_x': [forces['Propeller_3'][0] for forces in propeller_forces],
    'prop3_force_y': [forces['Propeller_3'][1] for forces in propeller_forces],
    'prop3_force_z': [forces['Propeller_3'][2] for forces in propeller_forces],
    'prop4_force_x': [forces['Propeller_4'][0] for forces in propeller_forces],
    'prop4_force_y': [forces['Propeller_4'][1] for forces in propeller_forces],
    'prop4_force_z': [forces['Propeller_4'][2] for forces in propeller_forces]
})

prop_forces_df.to_csv('propeller_forces.csv', index=False)

# Basic plotting to visualize results
plt.figure(figsize=(15, 10))

# Position plot
plt.subplot(2, 2, 1)
plt.plot(time_history, position_history[:, 0], label='X')
plt.plot(time_history, position_history[:, 1], label='Y')
plt.plot(time_history, position_history[:, 2], label='Z')
plt.xlabel('Time (s)')
plt.ylabel('Position (m)')
plt.title('Drone Position Over Time')
plt.legend()
plt.grid(True)

# Orientation plot
plt.subplot(2, 2, 2)
plt.plot(time_history, np.rad2deg(orientation_history[:, 0]), label='Roll')
plt.plot(time_history, np.rad2deg(orientation_history[:, 1]), label='Pitch')
plt.plot(time_history, np.rad2deg(orientation_history[:, 2]), label='Yaw')
plt.xlabel('Time (s)')
plt.ylabel('Angle (deg)')
plt.title('Drone Orientation Over Time')
plt.legend()
plt.grid(True)

# Force plot
plt.subplot(2, 2, 3)
plt.plot(time_history, force_history[:, 0], label='X')
plt.plot(time_history, force_history[:, 1], label='Y')
plt.plot(time_history, force_history[:, 2], label='Z')
plt.xlabel('Time (s)')
plt.ylabel('Force (N)')
plt.title('Total Forces Over Time')
plt.legend()
plt.grid(True)

# Moment plot
plt.subplot(2, 2, 4)
plt.plot(time_history, moment_history[:, 0], label='Roll')
plt.plot(time_history, moment_history[:, 1], label='Pitch')
plt.plot(time_history, moment_history[:, 2], label='Yaw')
plt.xlabel('Time (s)')
plt.ylabel('Moment (N·m)')
plt.title('Total Moments Over Time')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('open_loop_simulation_results.png')
plt.show()

print("Simulation complete. Results saved to CSV and PNG files.")

Propeller Loaded Successfully
Mesh Generated Successfully
